# How to: Extracting EMIT Spectra at Specified Coordinates

**Summary**  

In this notebook we will open a netCDF4 file from the Earth Surface Minteral Dust Source Investigation (EMIT) as an `xarray.Dataset`. We will then extract the spectra at point coordinates from a `.csv` as a dataframe, then save and plot the data.

**Requirements:**
+ A NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download EMIT data   
+ Selected the `emit_tutorials` environment as the kernel for this notebook.
  + For instructions on setting up the environment, follow the the `setup_instructions.md` included in the `/setup/` folder of the repository.  
+ Downloaded the necessary EMIT files to the `../data/` folder.
  + Instructions and a list of files can be found in the `setup_instructions.md` included in the `/setup/` folder of the repository. 

**Learning Objectives**
+ How to open an EMIT file as an `xarray.Dataset`
+ How to extract spectra at coordinates listed in a `.csv` file 

---

Import the required Python libraries.

In [ ]:
# Import Packages
import sys
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
sys.path.append('../modules/')
from emit_tools import emit_xarray

Next, set the path to an EMIT dataset as an object. In this example we use an L2A Reflectance file.

In [ ]:
fp = '/home/jovyan/shared/2023-emit-tutorials/data/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc'

Open the file downloaded and defined as `fp`. To do this, we will use the `emit_tools` module which contains a few helpful functions that can be used with EMIT data.

In [ ]:
ds = emit_xarray(fp)
ds

Now open the `.csv` included in the `/data/` directory as a `pandas.dataframe`.

> Note: The category values here are arbitrary and included as an example of an additional column users may want.

In [ ]:
points = pd.read_csv('../data/sample_coords.csv')
points

Set the `points` dataframe index as `ID` to utilize our existing point ID's as an index.

In [ ]:
points = points.set_index(['ID'])

Plot the points an a layer from the dataset to visualize where the points are located.

In [ ]:
ds.sel(bands=63).hvplot.image(cmap='greys', rasterize=True)*points.hvplot.scatter(x='Longitude',y='Latitude', by='ID', aspect='equal')

Convert the dataframe to an `xarray.Dataset`

In [ ]:
xp = points.to_xarray()
xp

Select the data from our EMIT dataset using the Latitude and Longitude coordinates from our point dataset, then convert the output to a `pandas.dataframe`.

In [ ]:
extracted = ds.sel(latitude=xp.Latitude,longitude=xp.Longitude, method='nearest').to_dataframe()
extracted

The output is a longform dataframe using the `'ID'` field as an index. This is missing our `'Category'` column from our original dataframe. Use the `pd.join` function to add the `'Category'` column to our dataset using `'ID'` as an index.

In [ ]:
df = extracted.join(points['Category'], on=['ID'])
df

Now we have a dataframe containing our initial data, in addition to the extracted point data. This a a good place to save an output as a `.csv`. Go ahead and do that below.

In [ ]:
df.to_csv('../data/example_out.csv')

We can use our dataframe to plot the reflectance data we extracted, but first, mask the reflectance values of `-0.01`, which represent deep water vapor absorption regions. To do this, assign values where the reflectance = `-0.01` to `np.nan`. 

In [ ]:
df.loc[:]['reflectance'][df.loc[:]['reflectance'] == -0.01] = np.nan

Plot the data using `hvplot`. We can use `by=` to separate the reflectances by their `ID`.

In [ ]:
df.hvplot(x='wavelengths',y='reflectance', by=['ID']).opts(xlabel='Wavelengths (nm)',ylabel='Reflectance')

---

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 01-23-2023  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.  